In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import csv
import pymongo
import time

In [2]:
sleep_time = 3

In [3]:
players_columns = ['player','Age','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%',
           'eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','year','place','team','team_abbr']
team_columns = ['team','G','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%',
           'FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','wins','losses','place','lat','lon','city','team_abbr']

In [4]:
def create_players_csv(players_data, year):
    file_name = 'players_data_' + year + '.csv'
    players_df = pd.DataFrame(players_data)
    players_df = players_df.rename(columns={'player':'name','2P':'Two_P','2P%':'Two_PP','2PA':'Two_PA','3P':'Three_P','3P%':'Three_PP','3PA':'Three_PA','FG%':'FGP','eFG%':'eFGP','FT%':'FTP'})
    players_df.replace('', 0, inplace=True)
    #convert statuses of teams from strings to numbers
    players_df = players_df.astype({'AST':int, 'BLK':int, 'DRB':int, 'FG':int, 'FGA':int, 'FGP':float, 'FT':int, 'FTA':int, 'FTP':float, 
                                'G':int, 'GS': int, 'MP':int,'ORB':int, 'PF':int, 'PTS':int, 'STL':int, 'TOV':int, 'TRB':int, 'Three_P':int, 
                                'Three_PA':int,'Three_PP':float, 'Two_P':int, 'Two_PA':int, 'Two_PP':float, 'eFGP':float,'Age': int})
    players_df.to_csv(file_name, index=False)

In [5]:
!which chromedriver
#for Mac
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

#for Windows
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

'which' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
players_ds = []
teams_ds = []

#if you have slow internet connection, change the list of "years" data from 10 years (['2010', ..., '2019']) 
# to a single year (ex: ['2010']) and run a single year each time
#In the end, it will create 10 csv files (players_data_2010.csv, players_data_2011.csv, ..., players_data_2019.csv)

#years = ['2010']
years = ['2010','2011','2012','2013','2014','2015','2016','2017', '2018', '2019']

with open('teams.csv', mode='r') as csv_file:
    csv_data = csv.DictReader(csv_file)
    for year in years:
        for team in csv_data:
            if year == '2010' or year == '2011' or year == '2012':
                if team['abbreviation'] == 'BRK':
                    url = "https://www.basketball-reference.com/teams/NJN/"+ year +".html"
                elif team['abbreviation'] == 'CHO':
                    url = "https://www.basketball-reference.com/teams/CHA/"+ year +".html"
                elif team['abbreviation'] == 'NOP':
                    url = "https://www.basketball-reference.com/teams/NOH/"+ year +".html"
                else:
                    url = "https://www.basketball-reference.com/teams/"+team['abbreviation']+"/"+ year +".html"
            elif year == '2013':
                if team['abbreviation'] == 'CHO':
                    url = "https://www.basketball-reference.com/teams/CHA/"+ year +".html"
                elif team['abbreviation'] == 'NOP':
                    url = "https://www.basketball-reference.com/teams/NOH/"+ year +".html"
                else:
                    url = "https://www.basketball-reference.com/teams/"+team['abbreviation']+"/"+ year +".html"
            elif year == '2014':
                if team['abbreviation'] == 'CHO':
                    url = "https://www.basketball-reference.com/teams/CHA/"+ year +".html"
                else:
                    url = "https://www.basketball-reference.com/teams/"+team['abbreviation']+"/"+ year +".html"
            else:
                url = "https://www.basketball-reference.com/teams/"+team['abbreviation']+"/"+ year +".html"
            
            browser.visit(url)
            time.sleep(sleep_time)

            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            players_table = soup.find('table', id="totals")
            players_tbody = players_table.find("tbody")
            players_trows = players_tbody.find_all("tr")

            row = {}

            for tr in players_trows:
                i = 0
                td = tr.find_all("td")
                for column in players_columns:
                    if column == 'year':
                        row[column] = year
                    elif column == 'place':
                        row[column] = team['place']
                    elif column == 'team':
                        row[column] = team['team']
                    elif column == 'team_abbr':
                        row[column] = team['abbreviation']
                    else:
                        row[column] = td[i].text
                        i += 1
                players_ds.append(dict(row))
        
        create_players_csv(players_ds, year)

    browser.quit()   

In [2]:
#load the 10 csv files into a dataframe (players_df)
data_years = ['2011','2012','2013','2014','2015','2016','2017', '2018', '2019']

players_df = pd.read_csv('players_data_2010.csv')

for data_year in data_years:
    csv_name = 'players_data_'+ data_year + '.csv'
    df = pd.read_csv(csv_name)
    players_df = pd.concat([players_df, df], ignore_index=True)

players_df

C:\Users\Kazuki\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,AST,Age,BLK,DRB,FG,FGA,FGP,FT,FTA,FTP,...,Three_PP,Two_P,Two_PA,Two_PP,eFGP,name,place,team,team_abbr,year
0,93,24,111,388,537,1143,0.470,181,234,0.774,...,0.372,416,818,0.509,0.523,Andrea Bargnani,Toronto,Raptors,TOR,2010
1,166,25,68,554,600,1158,0.518,470,590,0.797,...,0.364,592,1136,0.521,0.522,Chris Bosh,Toronto,Raptors,TOR,2010
2,304,30,32,301,276,674,0.409,175,226,0.774,...,0.374,168,385,0.436,0.490,Hedo Türkoğlu,Toronto,Raptors,TOR,2010
3,413,26,6,192,323,671,0.481,203,241,0.842,...,0.412,241,472,0.511,0.542,Jarrett Jack,Toronto,Raptors,TOR,2010
4,404,28,7,119,274,568,0.482,83,104,0.798,...,0.398,206,397,0.519,0.542,José Calderón,Toronto,Raptors,TOR,2010
5,53,20,18,157,255,512,0.498,148,194,0.763,...,0.250,251,496,0.506,0.502,DeMar DeRozan,Toronto,Raptors,TOR,2010
6,48,22,66,241,208,334,0.623,95,149,0.638,...,0.000,208,330,0.630,0.623,Amir Johnson,Toronto,Raptors,TOR,2010
7,71,25,11,152,162,399,0.406,53,77,0.688,...,0.335,104,226,0.460,0.479,Antoine Wright,Toronto,Raptors,TOR,2010
8,101,23,25,150,236,458,0.515,44,64,0.688,...,0.133,234,443,0.528,0.517,Sonny Weems,Toronto,Raptors,TOR,2010
9,89,23,5,81,154,379,0.406,101,121,0.835,...,0.380,94,221,0.425,0.485,Marco Belinelli,Toronto,Raptors,TOR,2010


In [3]:
#read the NBA salary data
salary_df = pd.read_csv('NBA_Full_Salaries_2000-2019.csv', index_col = 0)

In [4]:
#rename the column names to match the columns of players_df
salary_df = salary_df.rename(columns={'Name':'name','Year':'year'})
salary_df.head()

,name,year,Salaries,Rank
1,Shaquille O'Neal,2000,17142000.0,1
2,Kevin Garnett,2000,16806000.0,2
3,Alonzo Mourning,2000,15004000.0,3
4,Juwan Howard,2000,15000000.0,4
5,Scottie Pippen,2000,14795000.0,5


In [5]:
#merge players_df with salary_df
players_df = players_df.merge(salary_df, on=['name', 'year'], how='left')

In [6]:
players_df

,AST,Age,BLK,DRB,FG,FGA,FGP,FT,FTA,FTP,...,Two_PA,Two_PP,eFGP,name,place,team,team_abbr,year,Salaries,Rank
0,93,24,111,388,537,1143,0.470,181,234,0.774,...,818,0.509,0.523,Andrea Bargnani,Toronto,Raptors,TOR,2010,6527491.0,111.0
1,166,25,68,554,600,1158,0.518,470,590,0.797,...,1136,0.521,0.522,Chris Bosh,Toronto,Raptors,TOR,2010,15779912.0,20.0
2,304,30,32,301,276,674,0.409,175,226,0.774,...,385,0.436,0.490,Hedo Türkoğlu,Toronto,Raptors,TOR,2010,NaN,NaN
3,413,26,6,192,323,671,0.481,203,241,0.842,...,472,0.511,0.542,Jarrett Jack,Toronto,Raptors,TOR,2010,4500000.0,167.0
4,404,28,7,119,274,568,0.482,83,104,0.798,...,397,0.519,0.542,José Calderón,Toronto,Raptors,TOR,2010,NaN,NaN
5,53,20,18,157,255,512,0.498,148,194,0.763,...,496,0.506,0.502,DeMar DeRozan,Toronto,Raptors,TOR,2010,2282640.0,255.0
6,48,22,66,241,208,334,0.623,95,149,0.638,...,330,0.630,0.623,Amir Johnson,Toronto,Raptors,TOR,2010,3666666.0,197.0
7,71,25,11,152,162,399,0.406,53,77,0.688,...,226,0.460,0.479,Antoine Wright,Toronto,Raptors,TOR,2010,2110550.0,264.0
8,101,23,25,150,236,458,0.515,44,64,0.688,...,443,0.528,0.517,Sonny Weems,Toronto,Raptors,TOR,2010,736420.0,408.0
9,89,23,5,81,154,379,0.406,101,121,0.835,...,221,0.425,0.485,Marco Belinelli,Toronto,Raptors,TOR,2010,1547640.0,309.0


In [7]:
#drop nane in the players_df
players_df = players_df.dropna()

In [9]:
players_df

,AST,Age,BLK,DRB,FG,FGA,FGP,FT,FTA,FTP,...,Two_PA,Two_PP,eFGP,name,place,team,team_abbr,year,Salaries,Rank
0,93,24,111,388,537,1143,0.470,181,234,0.774,...,818,0.509,0.523,Andrea Bargnani,Toronto,Raptors,TOR,2010,6527491.0,111.0
1,166,25,68,554,600,1158,0.518,470,590,0.797,...,1136,0.521,0.522,Chris Bosh,Toronto,Raptors,TOR,2010,15779912.0,20.0
3,413,26,6,192,323,671,0.481,203,241,0.842,...,472,0.511,0.542,Jarrett Jack,Toronto,Raptors,TOR,2010,4500000.0,167.0
5,53,20,18,157,255,512,0.498,148,194,0.763,...,496,0.506,0.502,DeMar DeRozan,Toronto,Raptors,TOR,2010,2282640.0,255.0
6,48,22,66,241,208,334,0.623,95,149,0.638,...,330,0.630,0.623,Amir Johnson,Toronto,Raptors,TOR,2010,3666666.0,197.0
7,71,25,11,152,162,399,0.406,53,77,0.688,...,226,0.460,0.479,Antoine Wright,Toronto,Raptors,TOR,2010,2110550.0,264.0
8,101,23,25,150,236,458,0.515,44,64,0.688,...,443,0.528,0.517,Sonny Weems,Toronto,Raptors,TOR,2010,736420.0,408.0
9,89,23,5,81,154,379,0.406,101,121,0.835,...,221,0.425,0.485,Marco Belinelli,Toronto,Raptors,TOR,2010,1547640.0,309.0
11,8,29,4,75,34,69,0.493,27,60,0.450,...,69,0.493,0.493,Reggie Evans,Toronto,Raptors,TOR,2010,4960000.0,157.0
12,26,28,2,20,39,73,0.534,24,29,0.828,...,49,0.653,0.582,Marcus Banks,Toronto,Raptors,TOR,2010,4553793.0,165.0


In [8]:
players_df.to_csv('players_salary.csv', index=False)

In [2]:
#create a 'group' column and split data into 3 groups  by salary ranges (<=2,000,000; > 2,000,000 & <= 8,000,000; more than 8,000,000)
players_df['group'] = players_df['Salaries']
players_df['group'].loc[players_df['group']<=2000000] = 1
players_df['group'].loc[(players_df['group']<=8000000) & (players_df['group']>2000000)] = 2
players_df['group'].loc[players_df['group']>8000000] = 3
players_df

C:\Users\Kazuki\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,AST,Age,BLK,DRB,FG,FGA,FGP,FT,FTA,FTP,...,Two_PP,eFGP,name,place,team,team_abbr,year,Salaries,Rank,group
0,93,24,111,388,537,1143,0.470,181,234,0.774,...,0.509,0.523,Andrea Bargnani,Toronto,Raptors,TOR,2010,6527491.0,111.0,2.0
1,166,25,68,554,600,1158,0.518,470,590,0.797,...,0.521,0.522,Chris Bosh,Toronto,Raptors,TOR,2010,15779912.0,20.0,3.0
2,413,26,6,192,323,671,0.481,203,241,0.842,...,0.511,0.542,Jarrett Jack,Toronto,Raptors,TOR,2010,4500000.0,167.0,2.0
3,53,20,18,157,255,512,0.498,148,194,0.763,...,0.506,0.502,DeMar DeRozan,Toronto,Raptors,TOR,2010,2282640.0,255.0,2.0
4,48,22,66,241,208,334,0.623,95,149,0.638,...,0.630,0.623,Amir Johnson,Toronto,Raptors,TOR,2010,3666666.0,197.0,2.0
5,71,25,11,152,162,399,0.406,53,77,0.688,...,0.460,0.479,Antoine Wright,Toronto,Raptors,TOR,2010,2110550.0,264.0,2.0
6,101,23,25,150,236,458,0.515,44,64,0.688,...,0.528,0.517,Sonny Weems,Toronto,Raptors,TOR,2010,736420.0,408.0,1.0
7,89,23,5,81,154,379,0.406,101,121,0.835,...,0.425,0.485,Marco Belinelli,Toronto,Raptors,TOR,2010,1547640.0,309.0,1.0
8,8,29,4,75,34,69,0.493,27,60,0.450,...,0.493,0.493,Reggie Evans,Toronto,Raptors,TOR,2010,4960000.0,157.0,2.0
9,26,28,2,20,39,73,0.534,24,29,0.828,...,0.653,0.582,Marcus Banks,Toronto,Raptors,TOR,2010,4553793.0,165.0,2.0


In [3]:
#drop players who play less than 20 games
players_df = players_df.loc[players_df['G']>20]
players_df

,AST,Age,BLK,DRB,FG,FGA,FGP,FT,FTA,FTP,...,Two_PP,eFGP,name,place,team,team_abbr,year,Salaries,Rank,group
0,93,24,111,388,537,1143,0.470,181,234,0.774,...,0.509,0.523,Andrea Bargnani,Toronto,Raptors,TOR,2010,6527491.0,111.0,2.0
1,166,25,68,554,600,1158,0.518,470,590,0.797,...,0.521,0.522,Chris Bosh,Toronto,Raptors,TOR,2010,15779912.0,20.0,3.0
2,413,26,6,192,323,671,0.481,203,241,0.842,...,0.511,0.542,Jarrett Jack,Toronto,Raptors,TOR,2010,4500000.0,167.0,2.0
3,53,20,18,157,255,512,0.498,148,194,0.763,...,0.506,0.502,DeMar DeRozan,Toronto,Raptors,TOR,2010,2282640.0,255.0,2.0
4,48,22,66,241,208,334,0.623,95,149,0.638,...,0.630,0.623,Amir Johnson,Toronto,Raptors,TOR,2010,3666666.0,197.0,2.0
5,71,25,11,152,162,399,0.406,53,77,0.688,...,0.460,0.479,Antoine Wright,Toronto,Raptors,TOR,2010,2110550.0,264.0,2.0
6,101,23,25,150,236,458,0.515,44,64,0.688,...,0.528,0.517,Sonny Weems,Toronto,Raptors,TOR,2010,736420.0,408.0,1.0
7,89,23,5,81,154,379,0.406,101,121,0.835,...,0.425,0.485,Marco Belinelli,Toronto,Raptors,TOR,2010,1547640.0,309.0,1.0
8,8,29,4,75,34,69,0.493,27,60,0.450,...,0.493,0.493,Reggie Evans,Toronto,Raptors,TOR,2010,4960000.0,157.0,2.0
9,26,28,2,20,39,73,0.534,24,29,0.828,...,0.653,0.582,Marcus Banks,Toronto,Raptors,TOR,2010,4553793.0,165.0,2.0


In [5]:
#create a csv file "players_salary_by_group.csv"
players_df.to_csv("players_salary_by_groups.csv", index=False)